In [ ]:
import os
import cv2
from ultralytics import YOLO
import easyocr

In [ ]:
video_path = "E:\\My projects\\plaue detection(YOLOv8)\\plaue.mp4"
out_plaue = "frame"
os.makedirs(out_plaue,exist_ok=True)

In [ ]:
cap = cv2.VideoCapture(video_path)
frame_id = 0
while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f"{out_plaue}/frame_{frame_id:04d}.jpg",frame)
    frame_id += 1
cap.release()

In [ ]:
model = YOLO("yolov8n.pt")
model.train(data="E:\\My projects\\plaue detection(YOLOv8)\\data.yaml",epochs=100,imgsz=640,save=True,verbose=False)

In [ ]:
model.predict(data="E:\\My projects\\plaue detection(YOLOv8)\\runs\detect\\train\\weights\\best.pt",source="E:\\My projects\\plaue detection(YOLOv8)\\plaue.mp4",save=True)

In [ ]:
# بارگذاری مدل YOLOv8 آموزش‌دیده برای پلاک
model = YOLO("E:\\My projects\\plaue detection(YOLOv8)\\runs\\detect\\train\\weights\\best.pt")

# OCR reader
ocr = easyocr.Reader(["en"])

# بارگذاری ویدیو ورودی
video_path = "E:\\My projects\\plaue detection(YOLOv8)\\plaue.mp4"
cap = cv2.VideoCapture(video_path)

# تنظیمات ویدیو خروجی
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("output_with_plates.mp4", fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cropped = frame[y1:y2, x1:x2]

            # OCR روی ناحیه پلاک
            text = ocr.readtext(cropped)
            for t in text:
                plate_text = t[1]
                conf = t[2]
                if conf > 0.3:  # فقط خروجی‌های قابل اعتماد
                    cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

    out.write(frame)
    print(f"✅ Frame {frame_count} processed.")

cap.release()
out.release()
print("🎬 The output video was saved: output_with_plates.mp4")